## 라이브러리 호출

In [1]:
from urllib.request import Request, urlopen
from urllib.parse import unquote, urlencode, quote_plus

In [2]:
import xml.etree.ElementTree as ET
import pandas as pd

## 기본함수

### XML Parser

In [27]:
def df_from_xml(xtree):
    IsColListEmpty = True
    col_list = []
    df_list = []
    for tags in xtree:
        if(tags.tag == 'msgBody'):
            for busRouteList in tags:
                bus_route = []
                for col in busRouteList:
                    if(IsColListEmpty):
                        col_list.append(col.tag)
                    /print col.text
                    bus_route.append(col.text)
                IsColListEmpty = False
                df_list.append(bus_route)

    df = pd.DataFrame(df_list, columns=col_list)
    return df

## 노선번호를  넣으면 노선ID 목록을 반환한다.

In [4]:
class BusRouteService:
    def __init__(self, routeNo):
        self.routeNo = str(routeNo)
        _key = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D'
        self.key = unquote(_key)
        
    def getRouteId(self):
        url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice'
        queryParams = '?' + urlencode({quote_plus('serviceKey') : self.key, quote_plus('keyword') : self.routeNo })
        
        request = Request(url + queryParams)
        request.get_method = lambda: 'GET'
        raw_xml = urlopen(request).read().decode('utf8')
        
        xtree = ET.fromstring(raw_xml)      
        
        self.df = df_from_xml(xtree)
        
        return self.df
    
    def routeId(self, index):
        routeId = self.df.iloc[index]['routeId']
        return int(routeId)

In [5]:
routeNo = str(9004)
bus = BusRouteService(routeNo)
df_busrouteservice = bus.getRouteId()
index = 0
routeId = bus.routeId(index)

## 노선ID를 넣으면 정류소정보를 반환한다

In [28]:
class Station:
    def __init__(self, routeId):
        self.routeId = str(routeId)
        _key = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D'
        self.key = unquote(_key)
        
    def getStation(self):
        url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/station'
        queryParams = '?' + urlencode({quote_plus('serviceKey') : self.key, quote_plus('routeId') : self.routeId })
        
        request = Request(url + queryParams)
        request.get_method = lambda: 'GET'
        raw_xml = urlopen(request).read().decode('utf8')
        
        xtree = ET.fromstring(raw_xml)      
        
        self.df_station = df_from_xml(xtree)
        
        return self.df_station

In [29]:
station = Station(routeId)

In [30]:
df_station = station.getStation()

N
2
 07004
성남
206000010
운중동먹거리촌
127.0582333
37.3914
1
N
N
2
 07005
성남
206000009
운중동푸르지오하임
127.0611
37.3919333
2
N
N
2
 07581
성남
206000658
한빛교회.월든힐스아파트
127.0639667
37.39205
3
N
N
2
 07507
성남
206000558
운중초등학교
127.0665667
37.39135
4
N
N
2
 07009
성남
206000007
운중중학교
127.07305
37.3916
5
N
N
2
 07011
성남
206000006
운중동행정복지센터
127.0776667
37.3915
6
N
N
2
 07013
성남
206000005
뫼루니육교
127.0829167
37.3910667
7
N
N
2
 07015
성남
206000004
판교원마을1.2단지.판교도서관
127.0879333
37.3898167
8
N
N
2
 07502
성남
206000555
판교청소년수련관.판교종합사회복지관
127.0919333
37.3889833
9
N
N
2
 07016
성남
206000003
한림아파트
127.0961333
37.3882167
10
N
N
2
 07018
성남
206000002
너더리육교
127.0996
37.3881
11
N
N
2
성남
277103149
판교TG(경유)
127.1031667
37.39595
12
N
N
2
성남
277103679
금토JC(경유)
127.0853167
37.4152833
13
N
N
2
서울
277103441
서울진입(경유)
127.0665833
37.43385
14
N
N
2
서울
277104177
청계산입구역(경유)
127.0542
37.4483667
15
N
N
2
서울
277103388
양재IC(경유)
127.04045
37.4632833
16
N
N
2
서울
277103390
서초IC(경유)
127.0263333
37.4831333
17
N
N
2
 90062
서울
121000086
KCC사옥
127.02

In [26]:
df_station#[['x', 'y']]

,centerYn,districtCd,mobileNo,regionName,stationId,stationName,x,y,stationSeq,turnYn
0,N,2,07004,성남,206000010,운중동먹거리촌,127.0582333,37.3914,1,N
1,N,2,07005,성남,206000009,운중동푸르지오하임,127.0611,37.3919333,2,N
2,N,2,07581,성남,206000658,한빛교회.월든힐스아파트,127.0639667,37.39205,3,N
3,N,2,07507,성남,206000558,운중초등학교,127.0665667,37.39135,4,N
4,N,2,07009,성남,206000007,운중중학교,127.07305,37.3916,5,N
5,N,2,07011,성남,206000006,운중동행정복지센터,127.0776667,37.3915,6,N
6,N,2,07013,성남,206000005,뫼루니육교,127.0829167,37.3910667,7,N
7,N,2,07015,성남,206000004,판교원마을1.2단지.판교도서관,127.0879333,37.3898167,8,N
8,N,2,07502,성남,206000555,판교청소년수련관.판교종합사회복지관,127.0919333,37.3889833,9,N
9,N,2,07016,성남,206000003,한림아파트,127.0961333,37.3882167,10,N
